In [ ]:
import zipfile
from pathlib import Path
import shutil
from tqdm.auto import tqdm

In [ ]:
inputs = dict(snakemake.input)

In [ ]:
# Costs csv files are added without the absolute path to their zip file
# Reason: Historic, initial upload was like this and we keep the same structure
key = "costs"
fps = [Path(fp) for fp in inputs.pop(key)]
with zipfile.ZipFile(snakemake.output[key], mode="w") as ziparchive:
    for fp in fps:
        ziparchive.write(fp, arcname=fp.name) # arcname: add csv without basepath

In [ ]:
# results.csv is simply a copy of the file
key = "results_csv"
fp = inputs.pop(key)
shutil.copy(fp, snakemake.output[key])

In [ ]:
# Recursively add all files of paths which are directories
# Add paths as files if they are not directories
for key in inputs.copy().keys():
    print(f"Building zip for: {key}")
    paths = [Path(f) for f in inputs.pop(key)]
    with zipfile.ZipFile(snakemake.output[key], mode="w") as ziparchive:
        for path in paths:
            if path.is_dir():
                # All files in directory and subdirectory are to be added to zip
                fps = path.rglob("*")
            else:
                # path is file; convert to list for loop below to work
                fps = [path]
            
            for fp in tqdm(fps):
                ziparchive.write(fp)